In [1]:
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd

In [2]:
user = 'root'
password = input('please, enter mysql password: ')
host = 'localhost'
port = 3306
database = 'mavenfuzzyfactory'

def get_connected():
    engine = create_engine('mysql+mysqlconnector://{}:{}@{}:{}/{}'.format(user, password, host, port, database))
    return engine.connect()

if __name__=="__main__":
    try:
        engine = get_connected()
        print(f"CONNECTION TO THE {host} FOR USER {user} SUCCESSFULLY ESTABLISHED.")
    except Exception as e:
        print("CONNECTION COULD NOT BE ESTABLISHED DUE TO THE FOLLOWING ERROR: \n", e)

CONNECTION TO THE localhost FOR USER root SUCCESSFULLY ESTABLISHED.


1.	Gsearch seems to be the biggest driver of our business. Could you pull monthly 
trends for gsearch sessions and orders so that we can showcase the growth there? 

In [3]:
query = '''
SELECT
	YEAR(ws.created_at) AS year, 
    MONTH(ws.created_at) AS month, 
    COUNT(DISTINCT ws.website_session_id) AS sessions, 
    COUNT(DISTINCT o.order_id) AS orders, 
    COUNT(DISTINCT o.order_id)/COUNT(DISTINCT ws.website_session_id) AS conversion_rate
FROM website_sessions ws
	LEFT JOIN orders o
		ON o.website_session_id = ws.website_session_id
WHERE ws.created_at < '2012-11-27'
	AND ws.utm_source = 'gsearch'
GROUP BY 1,2;'''

engine.execute(query).fetchall()


[(2012, 3, 1860, 60, Decimal('0.0323')),
 (2012, 4, 3574, 92, Decimal('0.0257')),
 (2012, 5, 3410, 97, Decimal('0.0284')),
 (2012, 6, 3578, 121, Decimal('0.0338')),
 (2012, 7, 3811, 145, Decimal('0.0380')),
 (2012, 8, 4877, 184, Decimal('0.0377')),
 (2012, 9, 4491, 188, Decimal('0.0419')),
 (2012, 10, 5534, 234, Decimal('0.0423')),
 (2012, 11, 8889, 373, Decimal('0.0420'))]

In [4]:
df = pd.read_sql(query, engine)
print(df)

   year  month  sessions  orders  conversion_rate
0  2012      3      1860      60           0.0323
1  2012      4      3574      92           0.0257
2  2012      5      3410      97           0.0284
3  2012      6      3578     121           0.0338
4  2012      7      3811     145           0.0380
5  2012      8      4877     184           0.0377
6  2012      9      4491     188           0.0419
7  2012     10      5534     234           0.0423
8  2012     11      8889     373           0.0420


2.	Next, it would be great to see a similar monthly trend for Gsearch, but this time splitting out nonbrand 
and brand campaigns separately. I am wondering if brand is picking up at all. If so, this is a good story to tell. 


In [5]:
query = '''
SELECT
	YEAR(ws.created_at) AS year, 
    MONTH(ws.created_at) AS month, 
    COUNT(DISTINCT CASE WHEN utm_campaign = 'nonbrand' 
                    THEN ws.website_session_id 
                    ELSE NULL END) AS nonbrand_sessions, 
    COUNT(DISTINCT CASE WHEN utm_campaign = 'nonbrand' 
                    THEN o.order_id 
                    ELSE NULL END) AS nonbrand_orders,
    COUNT(DISTINCT CASE WHEN utm_campaign = 'brand' 
                    THEN ws.website_session_id 
                    ELSE NULL END) AS brand_sessions, 
    COUNT(DISTINCT CASE WHEN utm_campaign = 'brand' 
                    THEN o.order_id 
                    ELSE NULL END) AS brand_orders
FROM website_sessions ws
	LEFT JOIN orders o
		ON o.website_session_id = ws.website_session_id
WHERE ws.created_at < '2012-11-27'
	AND ws.utm_source = 'gsearch'
GROUP BY 1,2;
'''

engine.execute(query).fetchall()

[(2012, 3, 1852, 60, 8, 0),
 (2012, 4, 3509, 86, 65, 6),
 (2012, 5, 3295, 91, 115, 6),
 (2012, 6, 3439, 114, 139, 7),
 (2012, 7, 3660, 136, 151, 9),
 (2012, 8, 4673, 174, 204, 10),
 (2012, 9, 4227, 172, 264, 16),
 (2012, 10, 5197, 219, 337, 15),
 (2012, 11, 8506, 356, 383, 17)]

In [6]:
df = pd.read_sql(query, engine)
df

,year,month,nonbrand_sessions,nonbrand_orders,brand_sessions,brand_orders
0,2012,3,1852,60,8,0
1,2012,4,3509,86,65,6
2,2012,5,3295,91,115,6
3,2012,6,3439,114,139,7
4,2012,7,3660,136,151,9
5,2012,8,4673,174,204,10
6,2012,9,4227,172,264,16
7,2012,10,5197,219,337,15
8,2012,11,8506,356,383,17


3. While we’re on Gsearch, could you dive into nonbrand, and pull monthly sessions and orders split by device type? 
I want to flex our analytical muscles a little and show the board we really know our traffic sources. 

In [7]:
query = '''
SELECT
	YEAR(ws.created_at) AS year, 
    MONTH(ws.created_at) AS month, 
    COUNT(DISTINCT CASE WHEN device_type = 'desktop' 
                    THEN ws.website_session_id 
                    ELSE NULL END) AS desktop_sessions, 
    COUNT(DISTINCT CASE WHEN device_type = 'desktop' THEN 
                    o.order_id 
                    ELSE NULL END) AS desktop_orders,
    COUNT(DISTINCT CASE WHEN device_type = 'mobile' 
                    THEN ws.website_session_id 
                    ELSE NULL END) AS mobile_sessions, 
    COUNT(DISTINCT CASE WHEN device_type = 'mobile' 
                    THEN o.order_id 
                    ELSE NULL END) AS mobile_orders
FROM website_sessions ws
	LEFT JOIN orders o 
		ON o.website_session_id = ws.website_session_id
WHERE ws.created_at < '2012-11-27'
	AND ws.utm_source = 'gsearch'
    AND ws.utm_campaign = 'nonbrand'
GROUP BY 1,2;
'''

engine.execute(query).fetchall()

[(2012, 3, 1128, 50, 724, 10),
 (2012, 4, 2139, 75, 1370, 11),
 (2012, 5, 2276, 83, 1019, 8),
 (2012, 6, 2673, 106, 766, 8),
 (2012, 7, 2774, 122, 886, 14),
 (2012, 8, 3515, 165, 1158, 9),
 (2012, 9, 3171, 155, 1056, 17),
 (2012, 10, 3934, 201, 1263, 18),
 (2012, 11, 6457, 323, 2049, 33)]

In [8]:
df = pd.read_sql(query, engine)
print(df)

   year  month  desktop_sessions  desktop_orders  mobile_sessions  \
0  2012      3              1128              50              724   
1  2012      4              2139              75             1370   
2  2012      5              2276              83             1019   
3  2012      6              2673             106              766   
4  2012      7              2774             122              886   
5  2012      8              3515             165             1158   
6  2012      9              3171             155             1056   
7  2012     10              3934             201             1263   
8  2012     11              6457             323             2049   

   mobile_orders  
0             10  
1             11  
2              8  
3              8  
4             14  
5              9  
6             17  
7             18  
8             33  


4.	I’m worried that one of our more pessimistic board members may be concerned about the large % of traffic from Gsearch. 
Can you pull monthly trends for Gsearch, alongside monthly trends for each of our other channels?

In [9]:
# first, finding the various utm sources and referers to see the traffic we're getting

In [10]:
query = '''
SELECT DISTINCT 
	utm_source,
    utm_campaign, 
    http_referer
FROM website_sessions
WHERE website_sessions.created_at < '2012-11-27';
'''

engine.execute(query).fetchall()

[('gsearch', 'nonbrand', 'https://www.gsearch.com'),
 (None, None, None),
 ('gsearch', 'brand', 'https://www.gsearch.com'),
 (None, None, 'https://www.gsearch.com'),
 ('bsearch', 'brand', 'https://www.bsearch.com'),
 (None, None, 'https://www.bsearch.com'),
 ('bsearch', 'nonbrand', 'https://www.bsearch.com')]

In [11]:
df = pd.read_sql(query, engine)
print(df)

  utm_source utm_campaign             http_referer
0    gsearch     nonbrand  https://www.gsearch.com
1       None         None                     None
2    gsearch        brand  https://www.gsearch.com
3       None         None  https://www.gsearch.com
4    bsearch        brand  https://www.bsearch.com
5       None         None  https://www.bsearch.com
6    bsearch     nonbrand  https://www.bsearch.com


In [12]:
query = '''
SELECT
	YEAR(ws.created_at) AS year, 
    MONTH(ws.created_at) AS month, 
    COUNT(DISTINCT CASE WHEN utm_source = 'gsearch' 
                    THEN ws.website_session_id 
                    ELSE NULL END) AS gsearch_paid_sessions,
    COUNT(DISTINCT CASE WHEN utm_source = 'bsearch' 
                    THEN ws.website_session_id 
                    ELSE NULL END) AS bsearch_paid_sessions,
    COUNT(DISTINCT CASE WHEN utm_source IS NULL AND http_referer IS NOT NULL 
                    THEN ws.website_session_id 
                    ELSE NULL END) AS organic_search_sessions,
    COUNT(DISTINCT CASE WHEN utm_source IS NULL AND http_referer IS NULL 
                    THEN ws.website_session_id 
                    ELSE NULL END) AS direct_type_in_sessions
FROM website_sessions ws
	LEFT JOIN orders o 
		ON o.website_session_id = ws.website_session_id
WHERE ws.created_at < '2012-11-27'
GROUP BY 1,2;
'''

engine.execute(query).fetchall()

[(2012, 3, 1860, 2, 8, 9),
 (2012, 4, 3574, 11, 78, 71),
 (2012, 5, 3410, 25, 150, 151),
 (2012, 6, 3578, 25, 190, 170),
 (2012, 7, 3811, 44, 207, 187),
 (2012, 8, 4877, 705, 265, 250),
 (2012, 9, 4491, 1439, 331, 285),
 (2012, 10, 5534, 1781, 428, 440),
 (2012, 11, 8889, 2840, 536, 485)]

In [13]:
df = pd.read_sql(query, engine)
df

,year,month,gsearch_paid_sessions,bsearch_paid_sessions,organic_search_sessions,direct_type_in_sessions
0,2012,3,1860,2,8,9
1,2012,4,3574,11,78,71
2,2012,5,3410,25,150,151
3,2012,6,3578,25,190,170
4,2012,7,3811,44,207,187
5,2012,8,4877,705,265,250
6,2012,9,4491,1439,331,285
7,2012,10,5534,1781,428,440
8,2012,11,8889,2840,536,485


5.	I’d like to tell the story of our website performance improvements over the course of the first 8 months. 
Could you pull session to order conversion rates, by month? 

In [14]:
query = '''
SELECT
	YEAR(ws.created_at) AS year, 
    MONTH(ws.created_at) AS month, 
    COUNT(DISTINCT ws.website_session_id) AS sessions, 
    COUNT(DISTINCT o.order_id) AS orders, 
    COUNT(DISTINCT o.order_id) / COUNT(DISTINCT ws.website_session_id) AS conversion_rate    
FROM website_sessions ws
	LEFT JOIN orders o
		ON o.website_session_id = ws.website_session_id
WHERE ws.created_at < '2012-11-27'
GROUP BY 1,2;
'''

engine.execute(query).fetchall()

[(2012, 3, 1879, 60, Decimal('0.0319')),
 (2012, 4, 3734, 99, Decimal('0.0265')),
 (2012, 5, 3736, 108, Decimal('0.0289')),
 (2012, 6, 3963, 140, Decimal('0.0353')),
 (2012, 7, 4249, 169, Decimal('0.0398')),
 (2012, 8, 6097, 228, Decimal('0.0374')),
 (2012, 9, 6546, 287, Decimal('0.0438')),
 (2012, 10, 8183, 371, Decimal('0.0453')),
 (2012, 11, 12750, 561, Decimal('0.0440'))]

In [15]:
df = pd.read_sql(query, engine)
df

,year,month,sessions,orders,conversion_rate
0,2012,3,1879,60,0.0319
1,2012,4,3734,99,0.0265
2,2012,5,3736,108,0.0289
3,2012,6,3963,140,0.0353
4,2012,7,4249,169,0.0398
5,2012,8,6097,228,0.0374
6,2012,9,6546,287,0.0438
7,2012,10,8183,371,0.0453
8,2012,11,12750,561,0.0440


6.	For the gsearch lander test, please estimate the revenue that test earned us 
(Hint: Look at the increase in CVR from the test (Jun 19 – Jul 28), and use 
nonbrand sessions and revenue since then to calculate incremental value)

In [16]:
query = '''
SELECT
	MIN(website_pageview_id) AS first_test_pv
FROM website_pageviews
WHERE pageview_url = '/lander-1';
'''

engine.execute(query).fetchall()

[(23504,)]

In [19]:
def first_test_pageviews():
    try:
        query1 = '''
        -- for this step, we'll find the first pageview id 

        CREATE TEMPORARY TABLE first_test_pageviews
        SELECT
            wp.website_session_id, 
            MIN(wp.website_pageview_id) AS min_pageview_id
        FROM website_pageviews wp
            INNER JOIN website_sessions ws
                ON ws.website_session_id = wp.website_session_id
                AND ws.created_at < '2012-07-28' -- prescribed by the assignment
                AND wp.website_pageview_id >= 23504 -- first page_view
                AND utm_source = 'gsearch'
                AND utm_campaign = 'nonbrand'
        GROUP BY 
            wp.website_session_id;
'''
        engine.execute(query1)
    except:
        print('error')

first_test_pageviews()

In [21]:
query2 = '''SELECT * FROM first_test_pageviews;'''
df = pd.read_sql(query2, engine)
print(df)

      website_session_id  min_pageview_id
0                  11683            23504
1                  11684            23505
2                  11685            23506
3                  11686            23507
4                  11687            23509
...                  ...              ...
4572               16987            34991
4573               16988            34992
4574               16989            34996
4575               16990            34997
4576               16991            35000

[4577 rows x 2 columns]


In [22]:
def nonbrand_test_sessions_w_landing_pages():
    try:
        query3 = '''
        
        -- next, we'll bring in the landing page to each session, like last time, but restricting to home or lander-1 this time
        CREATE TEMPORARY TABLE nonbrand_test_sessions_w_landing_pages
        SELECT 
            ftp.website_session_id, 
            wp.pageview_url AS landing_page
        FROM first_test_pageviews ftp
            LEFT JOIN website_pageviews wp
                ON wp.website_pageview_id = ftp.min_pageview_id
        WHERE wp.pageview_url IN ('/home','/lander-1'); 
        '''
        engine.execute(query3)
    except:
        print('error')

nonbrand_test_sessions_w_landing_pages()

In [23]:
query4 = '''SELECT * FROM nonbrand_test_sessions_w_landing_pages;'''
df = pd.read_sql(query4, engine)
print(df)

      website_session_id landing_page
0                  11683    /lander-1
1                  11684        /home
2                  11685    /lander-1
3                  11686    /lander-1
4                  11687        /home
...                  ...          ...
4572               16987    /lander-1
4573               16988    /lander-1
4574               16989    /lander-1
4575               16990        /home
4576               16991    /lander-1

[4577 rows x 2 columns]


In [24]:
def nonbrand_test_sessions_w_orders():
    try:
        query5 = '''
        -- then we make a table to bring in orders
        CREATE TEMPORARY TABLE nonbrand_test_sessions_w_orders
        SELECT
            nts.website_session_id, 
            nts.landing_page, 
            o.order_id AS order_id

        FROM nonbrand_test_sessions_w_landing_pages nts
        LEFT JOIN orders o
            ON o.website_session_id = nts.website_session_id
        ;
        '''
        engine.execute(query5)
    except:
        print('error')

nonbrand_test_sessions_w_orders()

In [25]:
query6 = '''SELECT * FROM nonbrand_test_sessions_w_orders;'''
df = pd.read_sql(query6, engine)
print(df)

      website_session_id landing_page  order_id
0                  11683    /lander-1       NaN
1                  11684        /home       NaN
2                  11685    /lander-1       NaN
3                  11686    /lander-1       NaN
4                  11687        /home       NaN
...                  ...          ...       ...
4572               16987    /lander-1       NaN
4573               16988    /lander-1       NaN
4574               16989    /lander-1       NaN
4575               16990        /home       NaN
4576               16991    /lander-1       NaN

[4577 rows x 3 columns]


In [26]:
query7 = '''
SELECT
	landing_page, 
    COUNT(DISTINCT website_session_id) AS sessions, 
    COUNT(DISTINCT order_id) AS orders,
    COUNT(DISTINCT order_id) / COUNT(DISTINCT website_session_id) AS conversion_rate
FROM nonbrand_test_sessions_w_orders ntso
GROUP BY 1;
'''
engine.execute(query7).fetchall()
df = pd.read_sql(query7, engine)
print(df)

  landing_page  sessions  orders  conversion_rate
0        /home      2261      72           0.0318
1    /lander-1      2316      94           0.0406


In [28]:
# finding the most reent pageview for gsearch nonbrand where the traffic was sent to /home

query8 = '''
SELECT 
	MAX(ws.website_session_id) AS most_recent_gsearch_nonbrand_home_pageview 
FROM website_sessions ws
	LEFT JOIN website_pageviews wp
		ON wp.website_session_id = ws.website_session_id
WHERE utm_source = 'gsearch'
	AND utm_campaign = 'nonbrand'
    AND pageview_url = '/home'
    AND ws.created_at < '2012-11-27';
    '''
engine.execute(query8).fetchall()
df = pd.read_sql(query8, engine)
print(df)

   most_recent_gsearch_nonbrand_home_pageview
0                                       17145


In [29]:
query9 = '''
SELECT 
	COUNT(website_session_id) AS sessions_since_test
FROM website_sessions ws
WHERE created_at < '2012-11-27'
	AND website_session_id > 17145 -- last /home session
	AND utm_source = 'gsearch'
	AND utm_campaign = 'nonbrand'
;
'''
engine.execute(query9).fetchall()
df = pd.read_sql(query9, engine)
print(df)

   sessions_since_test
0                22972


7.	For the landing page test you analyzed previously, it would be great to show a full conversion funnel 
from each of the two pages to orders. You can use the same time period you analyzed last time (Jun 19 – Jul 28).

In [31]:
query1 = '''
SELECT
	ws.website_session_id, 
    wp.pageview_url, 
    wp.created_at AS pageview_created_at, 
    CASE WHEN pageview_url = '/home' THEN 1 ELSE 0 END AS homepage,
    CASE WHEN pageview_url = '/lander-1' THEN 1 ELSE 0 END AS custom_lander,
    CASE WHEN pageview_url = '/products' THEN 1 ELSE 0 END AS products_page,
    CASE WHEN pageview_url = '/the-original-mr-fuzzy' THEN 1 ELSE 0 END AS mrfuzzy_page, 
    CASE WHEN pageview_url = '/cart' THEN 1 ELSE 0 END AS cart_page,
    CASE WHEN pageview_url = '/shipping' THEN 1 ELSE 0 END AS shipping_page,
    CASE WHEN pageview_url = '/billing' THEN 1 ELSE 0 END AS billing_page,
    CASE WHEN pageview_url = '/thank-you-for-your-order' THEN 1 ELSE 0 END AS thankyou_page
FROM website_sessions ws
	LEFT JOIN website_pageviews wp
		ON ws.website_session_id = wp.website_session_id
WHERE ws.utm_source = 'gsearch' 
	AND ws.utm_campaign = 'nonbrand' 
    AND ws.created_at < '2012-07-28'
		AND ws.created_at > '2012-06-19'
ORDER BY 
	1,3;
'''
engine.execute(query1).fetchall()
df = pd.read_sql(query1, engine)
df

,website_session_id,pageview_url,pageview_created_at,homepage,custom_lander,products_page,mrfuzzy_page,cart_page,shipping_page,billing_page,thankyou_page
0,11683,/lander-1,2012-06-19 00:35:54,0,1,0,0,0,0,0,0
1,11684,/home,2012-06-19 00:48:25,1,0,0,0,0,0,0,0
2,11685,/lander-1,2012-06-19 01:02:38,0,1,0,0,0,0,0,0
3,11686,/lander-1,2012-06-19 01:31:57,0,1,0,0,0,0,0,0
4,11686,/products,2012-06-19 01:32:51,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9659,16989,/products,2012-07-27 23:13:45,0,0,1,0,0,0,0,0
9660,16990,/home,2012-07-27 23:13:25,1,0,0,0,0,0,0,0
9661,16990,/products,2012-07-27 23:19:00,0,0,1,0,0,0,0,0
9662,16990,/the-original-mr-fuzzy,2012-07-27 23:22:56,0,0,0,1,0,0,0,0


In [32]:
def session_level_made_it_flagged():
    try:
        query2 = '''
        CREATE TEMPORARY TABLE session_level_made_it_flagged
        SELECT
            website_session_id, 
            MAX(homepage) AS saw_homepage, 
            MAX(custom_lander) AS saw_custom_lander,
            MAX(products_page) AS product_made_it, 
            MAX(mrfuzzy_page) AS mrfuzzy_made_it, 
            MAX(cart_page) AS cart_made_it,
            MAX(shipping_page) AS shipping_made_it,
            MAX(billing_page) AS billing_made_it,
            MAX(thankyou_page) AS thankyou_made_it
        FROM(
        SELECT
            ws.website_session_id, 
            wp.pageview_url, 
            -- wp.created_at AS pageview_created_at, 
            CASE WHEN pageview_url = '/home' THEN 1 ELSE 0 END AS homepage,
            CASE WHEN pageview_url = '/lander-1' THEN 1 ELSE 0 END AS custom_lander,
            CASE WHEN pageview_url = '/products' THEN 1 ELSE 0 END AS products_page,
            CASE WHEN pageview_url = '/the-original-mr-fuzzy' THEN 1 ELSE 0 END AS mrfuzzy_page, 
            CASE WHEN pageview_url = '/cart' THEN 1 ELSE 0 END AS cart_page,
            CASE WHEN pageview_url = '/shipping' THEN 1 ELSE 0 END AS shipping_page,
            CASE WHEN pageview_url = '/billing' THEN 1 ELSE 0 END AS billing_page,
            CASE WHEN pageview_url = '/thank-you-for-your-order' THEN 1 ELSE 0 END AS thankyou_page
        FROM website_sessions ws
            LEFT JOIN website_pageviews wp
                ON ws.website_session_id = wp.website_session_id
        WHERE ws.utm_source = 'gsearch' 
            AND ws.utm_campaign = 'nonbrand' 
            AND ws.created_at < '2012-07-28'
                AND ws.created_at > '2012-06-19'
        ORDER BY 
            ws.website_session_id,
            wp.created_at
        ) AS pageview_level

        GROUP BY 
            website_session_id;

        '''
        engine.execute(query2)
    except:
        print('error')

session_level_made_it_flagged()

In [33]:
query3 = '''SELECT * FROM session_level_made_it_flagged;'''
engine.execute(query3).fetchall()
df = pd.read_sql(query3, engine)
df

,website_session_id,saw_homepage,saw_custom_lander,product_made_it,mrfuzzy_made_it,cart_made_it,shipping_made_it,billing_made_it,thankyou_made_it
0,11683,0,1,0,0,0,0,0,0
1,11684,1,0,0,0,0,0,0,0
2,11685,0,1,0,0,0,0,0,0
3,11686,0,1,1,0,0,0,0,0
4,11687,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
4572,16987,0,1,0,0,0,0,0,0
4573,16988,0,1,1,1,1,0,0,0
4574,16989,0,1,1,0,0,0,0,0
4575,16990,1,0,1,1,0,0,0,0


In [34]:
query4 = '''
-- then this would produce the final output, part 1
SELECT
	CASE 
		WHEN saw_homepage = 1 THEN 'saw_homepage'
        WHEN saw_custom_lander = 1 THEN 'saw_custom_lander'
        ELSE 'please check logic' 
	END AS segment, 
    COUNT(DISTINCT website_session_id) AS sessions,
    COUNT(DISTINCT CASE WHEN product_made_it = 1 THEN website_session_id ELSE NULL END) AS to_products,
    COUNT(DISTINCT CASE WHEN mrfuzzy_made_it = 1 THEN website_session_id ELSE NULL END) AS to_mrfuzzy,
    COUNT(DISTINCT CASE WHEN cart_made_it = 1 THEN website_session_id ELSE NULL END) AS to_cart,
    COUNT(DISTINCT CASE WHEN shipping_made_it = 1 THEN website_session_id ELSE NULL END) AS to_shipping,
    COUNT(DISTINCT CASE WHEN billing_made_it = 1 THEN website_session_id ELSE NULL END) AS to_billing,
    COUNT(DISTINCT CASE WHEN thankyou_made_it = 1 THEN website_session_id ELSE NULL END) AS to_thankyou
FROM session_level_made_it_flagged 
GROUP BY 1
;'''

engine.execute(query4).fetchall()
df = pd.read_sql(query4, engine)
df

,segment,sessions,to_products,to_mrfuzzy,to_cart,to_shipping,to_billing,to_thankyou
0,saw_custom_lander,2316,1083,772,348,231,197,94
1,saw_homepage,2261,942,684,296,200,168,72


In [35]:
query5 = '''
-- then this as final output part 2 - click rates

SELECT
	CASE 
		WHEN saw_homepage = 1 THEN 'saw_homepage'
        WHEN saw_custom_lander = 1 THEN 'saw_custom_lander'
        ELSE 'please check logic' 
	END AS segment, 
	COUNT(DISTINCT CASE WHEN product_made_it = 1 
                    THEN website_session_id 
                    ELSE NULL END) / COUNT(DISTINCT website_session_id) AS lander_click_rate,
    
    COUNT(DISTINCT CASE WHEN mrfuzzy_made_it = 1 
                    THEN website_session_id 
                    ELSE NULL END) / COUNT(DISTINCT CASE WHEN product_made_it = 1 
                                                        THEN website_session_id 
                                                        ELSE NULL END) AS products_click_rate,
    
    COUNT(DISTINCT CASE WHEN cart_made_it = 1 
                        THEN website_session_id 
                        ELSE NULL END) / COUNT(DISTINCT CASE WHEN mrfuzzy_made_it = 1 
                                                        THEN website_session_id 
                                                        ELSE NULL END) AS mrfuzzy_click_rate,
    
    COUNT(DISTINCT CASE WHEN shipping_made_it = 1 
                    THEN website_session_id 
                    ELSE NULL END) / COUNT(DISTINCT CASE WHEN cart_made_it = 1 
                                                    THEN website_session_id 
                                                    ELSE NULL END) AS cart_click_rate,
    
    COUNT(DISTINCT CASE WHEN billing_made_it = 1 
                        THEN website_session_id 
                        ELSE NULL END) / COUNT(DISTINCT CASE WHEN shipping_made_it = 1 
                                                        THEN website_session_id 
                                                        ELSE NULL END) AS shipping_click_rate,
    
    COUNT(DISTINCT CASE WHEN thankyou_made_it = 1 
                        THEN website_session_id 
                        ELSE NULL END) / COUNT(DISTINCT CASE WHEN billing_made_it = 1 
                                                            THEN website_session_id 
                                                            ELSE NULL END) AS billing_click_rate
FROM session_level_made_it_flagged
GROUP BY 1
;
'''
engine.execute(query5).fetchall()
df = pd.read_sql(query5, engine)
df

,segment,lander_click_rate,products_click_rate,mrfuzzy_click_rate,cart_click_rate,shipping_click_rate,billing_click_rate
0,saw_custom_lander,0.4676,0.7128,0.4508,0.6638,0.8528,0.4772
1,saw_homepage,0.4166,0.7261,0.4327,0.6757,0.8400,0.4286


8.	I’d love for you to quantify the impact of our billing test, as well. Please analyze the lift generated 
from the test (Sep 10 – Nov 10), in terms of revenue per billing page session, and then pull the number 
of billing page sessions for the past month to understand monthly impact.

In [36]:
def billing_page_session_monthly():
    try:
        query = '''
        SELECT
            billing_version_seen, 
            COUNT(DISTINCT website_session_id) AS sessions, 
            SUM(price_usd) / COUNT(DISTINCT website_session_id) AS revenue_per_billing_page_seen
        FROM( 
        SELECT 
            wp.website_session_id, 
            wp.pageview_url AS billing_version_seen, 
            o.order_id, 
            o.price_usd
        FROM website_pageviews wp
            LEFT JOIN orders o
                ON o.website_session_id = wp.website_session_id
        WHERE wp.created_at > '2012-09-10' 
            AND wp.created_at < '2012-11-10'
            AND wp.pageview_url IN ('/billing','/billing-2')
        ) AS billing_pageviews_and_order_data
        GROUP BY 1
        ;
        '''
        engine.execute(query).fetchall()
        df = pd.read_sql(query, engine)
        return df
    except:
        print('error')

billing_page_session_monthly()

,billing_version_seen,sessions,revenue_per_billing_page_seen
0,/billing,657,22.826484
1,/billing-2,654,31.339297


In [38]:
# -- $22.83 revenue per billing page seen for the old version
# -- $31.34 for the new version
# -- LIFT: $8.51 per billing page view

query1 = '''
SELECT 
	COUNT(website_session_id) AS billing_sessions_past_month
FROM website_pageviews 
WHERE website_pageviews.pageview_url IN ('/billing','/billing-2') 
	AND created_at BETWEEN '2012-10-27' AND '2012-11-27'; -- past month
'''

# -- 1,194 billing sessions past month
# -- LIFT: $8.51 per billing session
# -- VALUE OF BILLING TEST: $10,160 over the past month

engine.execute(query1).fetchall()
df = pd.read_sql(query1, engine)
df

Exception during reset or similar
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 682, in do_rollback
    dbapi_connection.rollback()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mysql/connector/connection_cext.py", line 419, in rollback
    self._cmysql.rollback()
_mysql_connector.MySQLInterfaceError: Commands out of sync; you can't run this command now


,billing_sessions_past_month
0,1193
